In [0]:
%python
df=spark.read.table("dev.harsha_bronze.sales")

df1=df.dropDuplicates().dropna().drop("ingestion_date")

df1.write.mode("overwrite").saveAsTable("dev.harsha_silver.sales_cleaned")

In [0]:
%sql
create or replace table dev.harsha_silver.products_scd1 (product_id int , product_name string, price double, category string)

In [0]:
%sql
with deduplicated_sources as 
(
  select s.*,
  row_number() over(partition by product_id order by seqNum desc) as row_number
  from dev.harsha_bronze.products s
  )
merge into dev.harsha_silver.products_scd1 t
using (select * from deduplicated_sources where row_number =1)s 
on t.product_id = s.product_id
when matched and s.operation="DELETE" 
then delete
when matched and s.operation="UPDATE" then update
set 
t.product_name = s.product_name,
t.price = s.product_price,
t.category = s.product_category
when not matched then insert 
(t.product_id, t.product_name, t.price, t.category) values (s.product_id, s.product_name, s.product_price, s.product_category)